In [4]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.utils.globus_utils import GlobusConfig, create_globus_tc
from multiplex_pipeline.utils.file_utils import globus_dir_exists, GlobusPathConverter
from multiplex_pipeline.core_cutting.file_io import GlobusFileStrategy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# experiment settings file path
settings_path = r'R:\Wayne\BLCA\BLCA-3_Analysis\analysis_settings_BLCA3.yaml'

# globus config
config_path = Path(r'D:\globus_config\globus_config.yaml')

In [10]:
# load analysis configuration
settings = load_analysis_settings(settings_path)

In [11]:
settings

{'image_dir': 'R:/CellDive/BLCA-3/BLCA-3_Final',
 'analysis_name': 'BLCA-3_Analysis',
 'local_analysis_dir': 'R:/Wayne/BLCA',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('R:/Wayne/BLCA/BLCA-3_Analysis/logs'),
 'detection_image': 'BLCA-3_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('R:/Wayne/BLCA/BLCA-3_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('R:/Wayne/BLCA/BLCA-3_Analysis/temp'),
 'cores_dir_output': WindowsPath('R:/Wayne/BLCA/BLCA-3_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': '

In [8]:
# get Globus settings
gc = GlobusConfig.from_config_files(config_path, from_collection = 'r_collection_id', to_collection = 'crcd_collection_id')
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [12]:
# check if the destination analysis directory exists 
remote_analysis_dir = settings["remote_analysis_dir"] + '/' + settings["analysis_name"]

globus_dir_exists(tc, gc.destination_collection_id, remote_analysis_dir)

True

In [13]:
# build a transfer map

# if transfering from Windows with multiple drives exposed in Globus use "multi_drive"
# if transfering from Windows with a single drive exposed in Globus 
# or if transfering from CIFs drive use "single_drive"
conv = GlobusPathConverter(layout="single_drive")

local_settings_path = conv.windows_to_globus(settings_path)
remote_settings_path = remote_analysis_dir + '/' + Path(settings_path).name
    
local_cores_info_path = conv.windows_to_globus(settings['core_info_file_path'].with_suffix('.pkl'))
remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

transfer_map = {}
transfer_map['settings'] = (local_settings_path, remote_settings_path)
transfer_map['core_definitions'] = (local_cores_info_path, remote_cores_info_path)
transfer_map

{'settings': ('/Wayne/BLCA/BLCA-3_Analysis/analysis_settings_BLCA3.yaml',
  '/ix1/kkedziora/blca_analysis/BLCA-3_Analysis/analysis_settings_BLCA3.yaml'),
 'core_definitions': ('/Wayne/BLCA/BLCA-3_Analysis/cores.pkl',
  '/ix1/kkedziora/blca_analysis/BLCA-3_Analysis/cores.pkl')}

In [14]:
# submit transfer
transfer = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = False)

2025-10-08 12:55:11.680 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:93 - Submitted transfer for settings to /ix1/kkedziora/blca_analysis/BLCA-3_Analysis/analysis_settings_BLCA3.yaml (task_id=8c3cea84-a467-11f0-ac3e-0affca67c55f)
2025-10-08 12:55:12.898 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:93 - Submitted transfer for core_definitions to /ix1/kkedziora/blca_analysis/BLCA-3_Analysis/cores.pkl (task_id=8d514710-a467-11f0-92dd-0affca67c55f)


In [17]:
# monitor transfer
print(f'Status of {transfer.pending[0][1]}: {tc.get_task(transfer.pending[0][0])["status"]}')
print(f'Status of {transfer.pending[1][1]}: {tc.get_task(transfer.pending[1][0])["status"]}')

Status of /ix1/kkedziora/blca_analysis/BLCA-3_Analysis/analysis_settings_BLCA3.yaml: SUCCEEDED
Status of /ix1/kkedziora/blca_analysis/BLCA-3_Analysis/cores.pkl: SUCCEEDED
